In [1]:
import numpy as np
import math
import random
from scipy.stats import mode
import matplotlib.pyplot as plt
from random import choices
from random import sample
import warnings
import datetime
warnings.filterwarnings('ignore')

# 1. Crowdsourcing Setup

In this part, we first define the parameters that will be used in our crowdsourcing model. Then, we simulate the reports for for all agents based on the crowdsourcing parameters.

In [2]:
"""
Define the crowdsourcing model and generate agents' reports.
"""

class Crowdsourcing:
    def __init__(self, w, Gamma_w, Gamma_s, 
                       m = 1000, 
                       n0 = 5, 
                       mi = 100, 
                       signal = [1,2,3,4,5]):
        self.m = m # Total number of tasks
        self.n0 = n0 # Minimum number of agents that are assigned to each task
        self.mi = mi # Number of tasks that each agent answers
        self.signal = signal # Signal space
        self.K = len(signal)
        self.w = w # Prior of ground truth
        self.Gamma_w = Gamma_w # Confusion matrix of full effort worker
        self.Gamma_s = Gamma_s # Confusion matrix of zero effort shirker
        
        
def Report_Generator(para, E, n_E):
    """
    Input: "para" is a class variable of Crowdsouring that scores the parameters;
           "E" is an array of effort which stores the types of effort level within the crowds;
           "n_E" is an array of integers which stores the number of agents of each of the effort levels in "E".
            
    Output: "X" is the n*m matrix of all agents' reports;
            "Y" is the ground truth of all tasks;
            "agent_e" is the effort level of all agents, which is scored as the index corresponds in the vector E
    """
            
    m = para.m
    n0 = para.n0
    mi = para.mi
    w = para.w
    Gamma_w = para.Gamma_w
    Gamma_s = para.Gamma_s
    signal = para.signal
    n = np.sum(n_E)
    
    agent_e = np.zeros(n) # Store the effort level of all agents. For example, agent_e[i] = j means the effort level of agent i is E[j].
    for i in range(len(n_E)-1):
        agent_e[np.sum(np.array(n_E)[0:i+1]):np.sum(np.array(n_E)[0:i+2])] = i+1
        
    Y = choices(list(range(1, len(w)+1)), w, k = m) # Randomly sample the ground truth of all tasks given prior w.
    
    """
    Loop each of the tasks and randomly assign n0 agents. If at the end this assignment does not work, redo everything.
    """
    flag = 0 # Mark the number of tasks that are assigned with at least n0 agents.
    while flag < m: # until all the tasks have been marked as finished.
        flag = 0
        current_task = np.zeros(n) # Dynamically store the number of tasks each agent has answered in the current round.
        X = np.zeros((n, m))
        for j, y in enumerate(Y):
            if np.count_nonzero(current_task < mi) < n0: # If the number of remaining agents who can still work is less than the minimum required number n0, then the current assignment does not work, rerun everything.
                break
            else:
                flag += 1 # there are enough agents to work, then one more task is finished.
            random_agents = np.array(sample(list(np.where(current_task < mi)[0]), n0)) # randomly select the agents who will work on the current task.
            for i in random_agents:
                e = E[int(agent_e[i])]
                Gamma_i = e*Gamma_w + (1-e)*Gamma_s
                X[i,j] = choices(signal, Gamma_i[Y[j]-1])[0]
            current_task[random_agents] += 1 # mark that the sellected agents have used one more of their mi budget.
    
    """
    For those agents who worked on less than mi tasks, randomly assign them some tasks to work.
    """
    extra_agents = np.where(np.count_nonzero(X, axis = 1) < mi)[0]
    for i in extra_agents:
        e = E[int(agent_e[i])]
        Gamma_i = e*Gamma_w + (1-e)*Gamma_s
        more_tasks = sample(list(np.where(X[i] == 0)[0]), mi - np.count_nonzero(X[i])) # find more tasks that has not been answered by agent i.
        for j in more_tasks:
            X[i,j] = choices(signal, Gamma_i[Y[j]-1])[0]
    
    return (X, Y, agent_e)

# 2. Performance Measurements

This part contains the implementations of all the spot-checking and peer prediction mechanisms that will be discussed. 

## 2.1 Spot-checking

### 2.1.1 The accuracy based spot-checking mechanism (SC-Acc)

The mechanism scores each agent their accuracy on the spot-checked tasks.

In [6]:
def spot_check_greedy(X, check_p):
    """
    Input: "X" is all agents' report matrix;
           "check_p" is the spot-checking probability.
    Output: "check" is the index of tasks that are checked.
    ------------------
    The algorithm selects the spot-checking tasks based on the number of disagreed answers.
    """
    m = np.size(X, axis = 1)
    disagree = np.zeros(m)
    for i in range(m):
        index = np.where(X[:,i] != 0)[0]
        maj = mode(X[index,i])[0]
        disagree[i] = np.count_nonzero(X[index,i] != maj)
    m_check = int(m*check_p)
    check = np.argsort(disagree)[-m_check:]
    return check

def spot_check_random(X, check_p):
    """
    Input: "X" is all agents' report matrix;
           "check_p" is the spot-checking probability.
    Output: "check" is the index of tasks that are checked.
    ------------------
    The algorithm randomly selects the spot-checking tasks.
    """
    m = np.size(X, axis = 1)
    m_check = int(m*check_p)
    check = np.array(random.sample(list(range(m)),m_check))
    return check


def Spot_check_mechanism_acc_w1(X, Y_gt):
    """
    Input: "X" is all agents' report matrix;
           "Y_gt" stores the ground truth of the spot-checked tasks. Unchecked tasks have values of zero.
    Output: "S" is the score of all agents.
    ------------------
    This mechanism is designed for W1 where the ground truth space is 2 and the signal space is 5.
    Agents' reports are projected down to the binary space.
    """
    n = np.size(X, axis = 0)
    m = np.size(X, axis = 1)
    Y_gt = np.array(Y_gt)
    index_Y = np.where(Y_gt != 0)[0]
    S = np.zeros(n)
    for i in range(n):
        X_projected = np.zeros(m)
        X_projected[X[i] == 1] = 1
        X_projected[X[i] == 2] = 1
        X_projected[X[i] == 4] = 2
        X_projected[X[i] == 5] = 2
        index_i = np.where(X_projected != 0)[0] # tasks ansewred by i
        index = np.intersect1d(index_i, index_Y) # tasks answered by i and checked
        S[i] = np.count_nonzero(X_projected[index] == Y_gt[index])/len(index)
    return S



def Spot_check_mechanism_acc_w2(X, Y_gt):
    """
    Input: "X" is all agents' report matrix;
           "Y_gt" stores the ground truth of the spot-checked tasks. Unchecked tasks have values of zero.
    Output: "S" is the score of all agents.
    """
    n = np.size(X, axis = 0)
    m = np.size(X, axis = 1)
    Y_gt = np.array(Y_gt)
    index_Y = np.where(Y_gt != 0)[0]
    S = np.zeros(n)
    for i in range(n):
        index_i = np.where(X[i] != 0)[0]
        index = np.intersect1d(index_i, index_Y)
        S[i] = np.count_nonzero(X[i][index] == Y_gt[index])/len(index)
    return S

### 2.1.2 The Dasgupta-Ghosh spot-checking mechanism (SC-DG)

Given an agent's reports and a set of spot-checking questions with the ground truth, SC-DG randomly chooses a common task (bonus task) and two distinct tasks (penalty tasks). Then, the agent is scored $1$ if her report on the bonus task agrees with the ground truth, and scored $-1$ if agreeing on the penalty tasks. The final score of the agent is the average score after repeated sampling. 

In [7]:
def Spot_check_mechanism_DG_w1(X, Y_gt):
    """
    Input: "X" is all agents' report matrix;
           "Y_gt" stores the ground truth of the spot-checked tasks. Unchecked tasks have values of zero.
    Output: "S" is the score of all agents.
    """
    n = np.size(X, axis = 0)
    m = np.size(X, axis = 1)
    Y_gt = np.array(Y_gt)
    T = np.size(np.where(Y_gt != 0)[0])*5 # number of repeated rounds
    index_Y = np.where(Y_gt != 0)[0]
    S = np.zeros(n)
    for i in range(n):
        X_projected = np.zeros(m)
        X_projected[X[i] == 1] = 1
        X_projected[X[i] == 2] = 1
        X_projected[X[i] == 4] = 2
        X_projected[X[i] == 5] = 2
        index_i = np.where(X_projected != 0)[0] # tasks ansewred by i
        index = np.intersect1d(index_i, index_Y) # tasks answered by i and checked
        for j in range(T):
            
            # sample bonus and penalty tasks
            b = sample(list(index), 1)[0]
            p = sample(list(index_i), 1)[0]
            while p == b:
                p = sample(list(index_i),1)[0]
            q = sample(list(index_Y), 1)[0]
            while q == b or q == p:
                q = sample(list(index_Y), 1)[0]
                
            if X_projected[b] == Y_gt[b]:
                S[i] += 1
            if X_projected[p] == Y_gt[q]:
                S[i] -= 1
    return S/T



def Spot_check_mechanism_DG_w2(X, Y_gt):
    """
    Input: "X" is all agents' report matrix;
           "Y_gt" stores the ground truth of the spot-checked tasks. Unchecked tasks have values of zero.
    Output: "S" is the score of all agents.
    """
    n = np.size(X, axis = 0)
    m = np.size(X, axis = 1)
    Y_gt = np.array(Y_gt)
    T = np.size(np.where(Y_gt != 0)[0])*5
    index_Y = np.where(Y_gt != 0)[0]
    S = np.zeros(n)
    for i in range(n):
        index_i = np.where(X[i] != 0)[0]
        index = np.intersect1d(index_i, index_Y)
        for j in range(T):
            b = sample(list(index), 1)[0]
            p = sample(list(index_i), 1)[0]
            while p == b:
                p = sample(list(index_i),1)[0]
            q = sample(list(index_Y), 1)[0]
            while q == b or q == p:
                q = sample(list(index_Y), 1)[0]
            if X[i][b] == Y_gt[b]:
                S[i] += 1
            if X[i][p] == Y_gt[q]:
                S[i] -= 1
    return S/T


## 2.2 Peer Prediction

### 2.2.1 The output agreement mechanism (OA)

OA pays an agent $1$ if her report on a random task agrees with a random peer's report on the same task, and paying $0$ otherwise. The score is then the averaged value while taking average over all the peers and tasks. To speed up the mechanism, instead of pairing agent $i$ with each of her peer $j$, we simply learn the empirical distributions of the reports on each task of all agents but $i$. This can be seen as a "virtual agent" reporting based on the empirical distributions of all agents but $i$. Then, agent $i$ is paired with only one agent: the virtual agent.

We note that we use the trick of virtual agent in all of the peer prediction mechanisms that we implemented.

In [8]:
def mechanism_OA_individual(X, para, i):
    """
    Input: "X" is all agents' report matrix;
           "para" contains the crowdsourcing parameters;
           "i" is the index of a particular agent.
    Output: "S" is the score of agent i computed with the OA mechanism.
    """
    n = len(X)
    X = np.array(X)
    X_ni = np.vstack((X[0:i], X[i+1:n])) # all agents' reports but i.
    pv = soft_predictor_learner(X_ni, para) # report distribution of the virtual agent (the empirical report distribution of all agents but i)
    index_i = np.where(X[i] != 0)[0]
    S = 0
    for j in index_i:
        S += pv[j,int(X[i,j])-1]/len(index_i)
    return S

def soft_predictor_learner(X, para):
    """
    Input: "X" is a matrix of agents' reports with each row the reports of each agent and each column the reports on each task.
           "para" contains the crowdsourcing parameters.
    Output: "P" is a m*K matrix where each row is the empirical distribution of agents' reports in X on each task.
    """
    K = para.K
    X = np.array(X)
    m = np.size(X, axis = 1)
    P = []
    for j in range(m):
        pj =  np.zeros(K)
        nj = np.count_nonzero(X[:,j])
        if nj != 0:
            for i in range(K):
                pj[i] = np.count_nonzero(X[:,j] == i+1)/nj # compute the empirical distribution of task j
        P.append(pj)
    return np.array(P)

### 2.2.2 The Peer Truth Serum (PTS)

The only difference between PTS and OA is that in PTS, the payment if two agents agree on a task is proportional to $\frac{1}{R(x)}$, where $R$ is a public distribution of reports and $x$ is the common report. We implement PTS by setting $R$ to be the empirical distribution of all the other agents' reports other than $i$ while computing agent $i$'s payment.

In [9]:
def mechanism_PTS_individual(X, para, i):
    """
    Input: "X" is all agents' report matrix;
           "para" contains the crowdsourcing parameters;
           "i" is the index of a particular agent.
    Output: "S" is the score of agent i computed with the PTS mechanism.
    """
    n = len(X)
    K = para.K
    X = np.array(X)
    X_ni = np.vstack((X[0:i], X[i+1:n]))
    pv = soft_predictor_learner(X_ni, para) # report distribution of the virtual agent (the empirical report distribution of all agents but i)
    index_i = np.where(X[i] != 0)[0]
    counts = np.bincount(X_ni.astype(int).reshape(-1), minlength = K+1)
    sig_prior = counts[1:]/np.sum(counts[1:]) # This is R, the empirical report distribution of all agents but i
    S = 0
    for j in index_i:
        S += pv[j,int(X[i,j])-1]/(sig_prior[int(X[i,j])-1]*len(index_i))
    return S

### 2.2.3 The $f$-matrix mutual information mechanism (f-MMI)

$f$-\textbf{MMI} scores each agent using the estimation of the $f$-mutual information between her reports and her peer's report, where $f$ can be any convex function. 

With attention to detail, the $f$-MMI uses the empirical distributions to estimate the mutual information. 
We can simply estimate the empirical distributions between two agents' reports, i.e.~$\widetilde{P}_{\hat{X}_i, \hat{X}_j}$ for joint distribution and $\widetilde{P}_{\hat{X}_i}$ for marginal distribution. Then, the MI between reports $\hat{X}_i$ and $\hat{X}_j$ can be estimated, 
\begin{equation}\label{eq:estimated_MI_matrix}
    \widetilde{MI}^{f-MMI}_{i,j} = \sum_{x,y}\widetilde{P}_{\hat{X}_i, \hat{X}_j}(x, y)f\left(\frac{\widetilde{P}_{\hat{X}_i}(x)\widetilde{P}_{\hat{X}_j}(y)}{\widetilde{P}_{\hat{X}_i, \hat{X}_j}(x, y)}\right).
\end{equation}

The matrix mutual information mechanism then scores each agent $i$ the average of the estimated MI between $i$ and each of her peer. To speed up the mechanism, we again use the "virtual agent" trick. Then, we learn the joint distribution as well as the mutual information between agent $i$'s reports and this virtual agent's reports.

We implement the following four types of $f$ functions.

   1) Total variation distance (TVD): $\frac{1}{2}|a-1|$ 
   
   2) KL-divergence (KL):   $a\log a$  
   
   3) Pearson $\chi^2$ (Sqr):   $(a-1)^2$ 
   
   4) Squared Hellinger (Hlg):   $\left(1-\sqrt{a}\right)^2$  

In [19]:
def mechanism_MMI_individual(X, f, para, i):
    """
    Input: "X" is all agents' report matrix;
           "f" is the index of the convex function for the f-MMI mechanism, f = 0, 1, 2, 3 corresponds to the four f functions that we implement;
           "para" contains the crowdsourcing parameters;
           "i" is the index of a particular agent.
    Output: "S" is the score of agent i computed with the f-MMI mechanism.
    """
    m = para.m
    n = np.size(X, axis = 0)

    X_ni = np.vstack((X[0:i], X[i+1:n]))
    pv = soft_predictor_learner(X_ni, para) # report distribution of the virtual agent (the empirical report distribution of all agents but i)
    Qv = np.sum(pv, axis = 0)/m # Marginal distribution of the "virtual agent".
    P, Qi = distribution_learner(X[i], pv, para) # P is the joint distribution, Q_i is the marginal distribution of agents i.
    S = MI_computer_MMI(P, Qi, Qv, f)
    return S

def distribution_learner(Xi, Xv, para): 
    """
    Input: "Xi" is the reports of an agent i;
           "Xv" is the report of the "virtual agent", i.e.~a probability distribution of reports on each task;
           "para" contains the crowdsourcing parameters.
    Output: "P" is the estimated joint distribution between agent i and the virtual agent's reports;
            "Qi" is the empirical marginal distribution of agent i's reports.
    """
    K = para.K
    index_i = np.where(Xi != 0)[0]
    P = np.zeros((K,K))
    for j in index_i:
        P[int(Xi[j])-1]+=Xv[j]/len(index_i)
    Qi = np.zeros(K)
    for j in range(K):
        Qi[j] = np.count_nonzero(Xi == j+1)/len(index_i)
    return P, Qi


def MI_computer_MMI(P, Q1, Q2, f):
    """
    Input: "P" is the joint distribution (matrix of size K*K);
           "Q1" is the marginal distribution of the row agent (vector of length K);
           "Q2" is the marginal distribution of the column agent (vector of length K);
           "f" is a convex function
    Output: "MI" is the estimated mutual information
    """
    Q = Q2*Q1.reshape(-1, 1) # Product of the marginal sitributions (size of K*K)
    
    if f == 0: # TVD
        MI = np.sum(np.absolute(P - Q))
    elif f == 1: # KL
        t = P*np.log(P/Q)
        nan_ind = np.isnan(t)
        t[nan_ind] = 0 
        MI = np.sum(t)
    elif f == 2: # Sqr
        t = P*(np.square((Q/P)-1))
        nan_ind = np.isnan(t)
        t[nan_ind] = 0 
        MI = np.sum(t)
    elif f == 3: # Hlg
        t = P*(np.square(np.sqrt(Q/P)-1))
        nan_ind = np.isnan(t)
        t[nan_ind] = 0 
        MI = np.sum(t)
    
    return MI


### 2.2.4 The $f$-pairing mutual information mechanism (f-PMI)

Again, the main idea is to estimate the mutual information between agents' reports. Difference from MMI, PMI applies a different way to estimate the mutual information. Note that mutual information is a function of the quotient of the joint distribution and the product of the marginal distributions. We name this ratio as the "joint to marginal product ratio", denoted as JP.
Then, JP can be further rewritten as:

\begin{align}
    \frac{P_{\hat{X}_i, \hat{X}_{-i}}(\hat{x}_i, \hat{x}_{-i})}{P_{\hat{X}_i}(\hat{x}_i)P_{\hat{X}_{-i}}(\hat{x}_{-i})}=\frac{P_{\hat{X}_i| \hat{X}_{-i}}(\hat{x}_i| \hat{x}_{-i})}{P_{\hat{X}_i}(\hat{x}_i)}.
\end{align}

The denominator can be empirically estimated. While the numerator is a soft-predictor, which, given the reports of all agents except $i$ on a particular task $j$, produces a forecast of agent $i$'s report on the same task in the form of a distribution. In our experiments, we set the soft-predictor for agent $i$'s report on task $j$ as the empirical distribution of all the other agents' reports on the same task. 

The mechanism randomly and equally divide the set of tasks into two subsets, and use one subset to learn JP and the other to evaluate and pay agents. Specifically, say JP is learned with set A. Then, the mechainsm randomly chooses a common task (bonus task, b) and two distinct tasks (penalty tasks, p and q) from set B. Each agent will then be rewarded based on the following formula: (The selection of bonus and penalty tasks is repeated for $T$ times and taking average.)
\begin{align}
    S_i = \sum_{\sigma\in\Sigma}p_{v,b}(\sigma)K(\hat{x}_{i,b}, \sigma) - \sum_{\sigma\in\Sigma}p_{v,q}(\sigma)f^*(K(\hat{x}_{i,p}, \sigma)),
\end{align}

where $p_v$ is the report distribution of the "virtual agent" which is a $m*K$ matrix, then $p_{v,b}(\sigma)$ is the empirical probability that the virtual agent (all agents but i) report $\sigma$ on task $b$. $\hat{x}_{i,b}$ is the report of agent i on task b. More importantly, $K = \partial f(JP)$ is a scoring function that depends on the input function $f$. Furthermore, $f^*$ is the convex conjugate of function $f$.

   1) TVD: $f(a) = \frac{1}{2}|a - 1|$; $\quad f^*(b) = b$ if $|b| <= 1/2$, otherwise $f^*(b) = \infty$; $\quad\partial f(a) = \frac{1}{2}$ if $a > 1$, $-\frac{1}{2}$ if $a < 1$, $[-\frac{1}{2},\frac{1}{2}]$ if $a = 1$.
   
   2) KL:   $f(a) = a\log a$; $\quad f^*(b) = \exp(b-1)$; $\quad \partial f(a) = 1 + \log (a)$.
   
   3) Sqr:   $f(a) = (a-1)^2$; $quad f^*(b) = 1 + \frac{1}{4}b^2$; $\quad \partial f(a) = 2a$.
   
   4) Hlg:   $f(a) = \left(1-\sqrt{a}\right)^2$; $\qquad f^*(b) = \frac{b}{1-b}$; $\quad\partial f(a) = 1 - \frac{1}{\sqrt(a)}$.

In [11]:
def mechanism_PMI_individual(X, f, para, i):
    """
    Input: "X" is all agents' report matrix;
           "f" is the index of the convex function for the f-PMI mechanism, f = 0, 1, 2, 3 corresponds to the four f functions that we implement;
           "para" contains the crowdsourcing parameters;
           "i" is the index of a particular agent.
    Output: "S" is the score of agent i computed with the f-PMI mechanism.
    """
    n = np.size(X, axis = 0)
    m = para.m
    X_ni = np.vstack((X[0:i], X[i+1:n]))
    S = PMI_learner(X_ni, X[i], f, para)

    return S

def PMI_learner(X, Xi, f, para):
    """
    Input: "X" is a matrix of agents' reports (all agents but i);
           "Xi" is a vector of agents i's reports;
           "f" is the index of the convex function;
           "para" contains the crowdsourcing parameters;
    Output: "S" is the score of agent i computed with the f-PMI mechanism.
    """
    m = para.m
    index_i = np.where(Xi != 0)[0]
    T = np.size(np.where(Xi != 0)[0])*3
    m_i = len(index_i)
    K = para.K
    
    # Seperating the tasks. We guarantee the tasks that agent i answers are equally divided.
    index_i_train, index_i_test = np.array_split(index_i, 2)
    index_X_train, index_X_test = np.array_split(np.where(Xi == 0)[0], 2)
    index_train = np.union1d(index_X_train, index_i_train)
    index_test = np.union1d(index_X_test, index_i_test)
    
    Xi_int = Xi.copy().astype(int)
    pv = soft_predictor_learner(X,para) # learn the report distribution of the virtual agent
    Qv = np.average(pv[index_train], axis = 0) # estimate the empirical marginal distribution of the virtual agent
    P, Qi = distribution_learner(Xi[index_train], pv[index_train], para) # estimate the empirical joint and marginal distribution
    
    S = 0
    for j in range(T): # repeat T times and randomly sample the bonus/penalty tasks
        b,q = sample(list(index_i_test), 2)
        p = sample(list(index_test), 1)[0]
        while p == b or p == q:
            p = sample(list(index_test),1)[0]

        Mi = MI_computer_PMI(Qi, Qv, P, pv, b, p, q, Xi_int, f)
        S += Mi
    S = S/(T)

    return S

def MI_computer_PMI(Qi, Qv, P, pv, b, p, q, Xi, f):
    """
    Input: "Qi" is the marginal distribution of an agent i;
           "Qv" is the marginal distribution of the virtual agent, i.e. the empirical marginal distribution of all agents' report but i;
           "P" is the joint distribution between agent i and the virtual agent;
           "pv" is the report distribution of the virtual agent on each task;
           "b" is the index of the bonus task;
           "p" and "q" are the index of the penalty tasks;
           "Xi" is agent i's reports;
           "f" is the index of the convex function.
    Output: "Mi" is estimated mutual information between agent i and the virtual agent.
    """
    K = len(Qi) # The size of the signal space
    Mi = 0
    K_b = 0 # The value of the scoring function K given the bonus task
    K_p = 0 # The value of the scoring function K given the penalty tasks
    predict_b = pv[b] # The prediction of the learned soft predictor on task b
    predict_p = pv[p] 
    
    # Compute the score on the bonus task
    product_marginal = np.outer(Qi, Qv)
    if Qi[Xi[b]-1] != 0: 
        Jp_b = P[Xi[b]-1]/product_marginal[Xi[b]-1] # The value of the joint to marginal product ratio given bonus task
        K_b_dist = np.zeros(K)
        if f == 0:
            K_b_dist[np.where(Jp_b > 1)[0]] = 0.5
            K_b_dist[np.where(Jp_b < 1)[0]] = -0.5
        elif f == 1:
            K_JP = 1 + np.log(Jp_b)
            K_b_dist = K_JP.copy()
            K_b_dist[np.where(K_JP > 10)[0]] = 10 # To avoid outliers, we truncate JP at +- 10
            K_b_dist[np.where(K_JP < -10)[0]] = -10
        elif f == 2:
            K_b_dist = 2*(Jp_b - 1)
        elif f == 3:
            K_JP = 1 - 1/np.sqrt(Jp_b)
            K_b_dist = K_JP.copy()
            K_b_dist[np.where(K_JP < -10)[0]] = -10
        K_b = np.sum(pv[b]*K_b_dist)
        
    # Compute the score on the penalty tasks and get the estimated mutual information
    if Qi[Xi[q]-1] != 0:
        Jp_p = P[Xi[q]-1]/product_marginal[Xi[q]-1] # The value of the joint to marginal product ratio given a penalty task
        K_p_dist = np.zeros(K)
        if f == 0:
            K_p_dist[np.where(Jp_p > 1)[0]] = 0.5
            K_p_dist[np.where(Jp_p < 1)[0]] = -0.5
            K_p = np.sum(pv[p]*K_p_dist)
            Mi = K_b - K_p
            
        elif f == 1:
            K_JP = 1 + np.log(Jp_p)
            K_p_dist = K_JP.copy()
            K_p_dist[np.where(K_JP > 10)[0]] = 10 # To avoid outliers, we truncate JP at +- 10
            K_p_dist[np.where(K_JP < -10)[0]] = -10
            K_p = np.sum(pv[p]*K_p_dist)
            Mi = K_b - np.exp(K_p-1)
            
        elif f == 2:
            K_p_dist = 2*(Jp_p - 1)
            K_p = np.sum(pv[p]*K_p_dist)
            Mi = K_b - np.square(K_p)/4 - K_p
            
        elif f == 3:
            K_JP = 1 - 1/np.sqrt(Jp_p)
            K_p_dist = K_JP.copy()
            K_p_dist[np.where(K_JP < -10)[0]] = -10
            K_p = np.sum(pv[p]*K_p_dist)
            Mi = K_b - K_p/(1-K_p)
    
    return Mi


### 2.2.5 The determinant mutual information mechanim (DMI)

Again, the goal is to reward each agent based on the estimated mutual information between agents' reports. 
Determinant mutual information is a generalized version of Shannon mutual information. Specifically, for a pair of agents $i$ and $j$, the set of the commonly answered tasks is divided into two disjoint subsets $A$ and $B$. Again, we empirically estimate the joint distribution with reports in $A$ and $B$ respectively, and score agent $i$ the product of the determinants of these two estimated joint distribution matrices and take average over all the other agents. Again, we implement the trick of the virtual agent.


In [12]:
def mechanism_DMI_individual(X, para, i):
    """
    Input: "X" is all agents' report matrix;
           "para" contains the crowdsourcing parameters;
           "i" is the index of a particular agent.
    Output: "S" is the score of agent i computed with the DMI mechanism.
    """
    n = np.size(X, axis = 0)
    mi = para.mi
    X_ni = np.vstack((X[0:i], X[i+1:n]))
    pv = soft_predictor_learner(X_ni, para) # report distribution of the virtual agent (the empirical report distribution of all agents but i)
    
    # Separate agent i's reports into two sets. Because the tasks are randomly sampled, we simply separate based on the first half and the second half.
    Xi_A = X[i].copy()
    Xi_A[np.where(X[i] != 0)[0][0:int(mi/2)]] = 0
    Xi_B = X[i].copy()
    Xi_B[np.where(X[i] != 0)[0][int(mi/2):mi]] = 0

    # Learn the joint distribution with reports in A and B respectively.
    PA,_ = distribution_learner(Xi_A, pv, para)
    PB,_ = distribution_learner(Xi_B, pv, para)

    S = np.linalg.det(PA)*np.linalg.det(PB)
    return S



# 3. Running the Performance Measurements

This part will run the performance measurements to generate samples of the performance scores under different settings. In particular, 3.1 will let one agent deviate to a slightly higher effort so as to learn the equilibrium effort; 3.2 will let one agent deviate to an untruthful strategy so as to test the truthfulness robustness.

## 3.1 Effort Deviation

### 3.1.1 Spot-checking 

In [ ]:
"""
Running spot-checking mechanisms in W1
---------------------------------
There are 2 types of spot-checking mechanisms and 1 parameter ("check_prob") to control the checking probability.
One can switch between these two mechanisms by comment/uncomment one at a time.
"""
w_1 = np.array([0.612903, 0.387097])
Gamma_w1 = np.array([[0.6842, 0.2211, 0.0316, 0.0368, 0.0263], [0.0917, 0.1916, 0.05, 0.2, 0.4667]])
Gamma_random = np.ones((2,5))/5

m = 1000
mi = 100
n0 = 5
n = 52 # Given m = 1000, mi = 100, n0 = 5, n should be at least 50. We provide to extra agents in favor of the random assignment process.
T = 100 # In total, each setting will be run T times and generate n*T = 5200 samples of performance scores.
Effort_list = np.arange(0,1,0.02) # The list of xi
S_samples = np.zeros((len(Effort_list),n*T))
check_prob = 0.2 # Probability of spot-checking
for j,xi in enumerate(Effort_list):
    print(xi)
    for t in range(T):
        para_w = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para_w, [xi], [n])
        Y_check = np.array(Y.copy())
        index = spot_check_random(X, check_prob)
        index_cover = np.setdiff1d(np.array(range(m)), index)
        Y_check[index_cover] = 0
        
        """
        Switch the mechanisms
        """
        S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_acc_w1(X,Y_check)
#         S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_DG_w1(X,Y_check)

"""
Save the samples as .npy directly.
"""
np.save('Samples/Effort_equilibrium/W1/Spot_check_Acc/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W1/Spot_check_DG/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)



In [ ]:
"""
Running spot-checking mechanisms in W2
---------------------------------
There are 2 types of spot-checking mechanisms and 1 parameter ("check_prob") to control the checking probability.
One can switch between these two mechanisms by comment/uncomment one at a time.
"""
w_w2 = np.array([0.19587629, 0.24054983, 0.24742268, 0.3161512])
Gamma_w2 = np.array([[0.77056673, 0.12157221, 0.08409506, 0.023766],
                 [0.09083969, 0.7351145, 0.12977099, 0.04427481],
                 [0.03326256, 0.06157113, 0.86624204, 0.03892427],
                 [0.06785509, 0.16388729, 0.09890742, 0.6693502]])
Gamma_random = np.ones((4,4))/4

m = 1000
mi = 100
n0 = 5
n = 52 # Given m = 1000, mi = 100, n0 = 5, n should be at least 50. We provide to extra agents in favor of the random assignment process.
T = 100 # In total, each setting will be run T times and generate n*T = 5200 samples of performance scores.
Effort_list = np.arange(0,1,0.02) # The list of xi
S_samples = np.zeros((len(Effort_list),n*T))
check_prob = 0.2 # Probability of spot-checking
for j,xi in enumerate(Effort_list):
    print(xi)
    for t in range(T):
        para_w = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal=[1,2,3,4])
        X, Y, agent_e = Report_Generator(para_w, [xi], [n])
        Y_check = np.array(Y.copy())
        index = spot_check_random(X, check_prob)
        index_cover = np.setdiff1d(np.array(range(m)), index)
        Y_check[index_cover] = 0
        
        """
        Switch the mechanisms
        """
        S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_acc_w2(X,Y_check)
#         S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_DG_w2(X,Y_check)

"""
Save the samples as .npy directly.
"""
np.save('Samples/Effort_equilibrium/W2/Spot_check_Acc/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W2/Spot_check_DG/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)



### 3.1.2 Peer prediction

In [20]:
"""
Running peer prediction mechanisms in W1
---------------------------------
There are 5 types of peer prediction mechanisms and 2 of them have additional 4 types of f functions.
Totally, there are 11 mechanisms. 
To run all of them, simply change "f" while running "mechanism_matrix_individual()" and "mechanism_pairing_individual()"
and one can switch to different mechanisms by comment/uncomment one mechanism at a time.
"""
w_w1 = np.array([0.612903, 0.387097])
Gamma_w1 = np.array([[0.6842, 0.2211, 0.0316, 0.0368, 0.0263], [0.0917, 0.1916, 0.05, 0.2, 0.4667]])
Gamma_random = np.ones((2,5))/5

"""
Switch the f functions
"""
f = 0 

m = 1000
mi = 100
n0 = 5
n = 52 # Given m = 1000, mi = 100, n0 = 5, n should be at least 50. We provide to extra agents in favor of the random assignment process.
T = 100 # In total, each setting will be run n*T times and generate 5200 samples of performance scores.
Effort_list = np.arange(0,0.97,0.02) # The list of xi
S_samples = np.zeros((2,len(Effort_list),n*T)) # The first dimension is before/after an unilateral deviation; the second dimention is the equilibrium effort xi; the third dimension is the samples.
Mechanisms = ['tvd','kl','sqr','hlg'] # Four f functions

"""
First, generate samples when everyone exerts effort xi
------------------------
Because for peer prediction, the score distribution after a unilateral deviation is not equal to the score distribution
where everyone deviates to a higher effort, we have to generate the scores separately. 
"""
for j,xi in enumerate(Effort_list):
    print('before deviation: ', xi, datetime.datetime.now())
    for t in range(n*T):
        para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
        i = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[0,j,t] = mechanism_MMI_individual(X, f, para, i)
#         S_samples[0,j,t] = mechanism_PMI_individual(X, f, para, i)
#         S_samples[0,j,t] = mechanism_DMI_individual(X, para, i)
#         S_samples[0,j,t] = mechanism_OA_individual(X, para, i)
#         S_samples[0,j,t] = mechanism_PTS_individual(X, para, i)

"""
Then, generate samples when one agent deviates to effort xi+de
"""
for j,xi in enumerate(Effort_list):
    print('after deviation: ', xi, datetime.datetime.now())
    for t in range(n*T):
        para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para, [xi,xi+0.04], [n-1,1])
        i = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[1,j,t] = mechanism_MMI_individual(X, f, para, i)
#         S_samples[1,j,t] = mechanism_PMI_individual(X, f, para, i)
#         S_samples[1,j,t] = mechanism_DMI_individual(X, para, i)
#         S_samples[1,j,t] = mechanism_OA_individual(X, para, i)
#         S_samples[1,j,t] = mechanism_PTS_individual(X, para, i)
       
"""
Save the samples as .npy directly.
"""
np.save('Samples/Effort_equilibrium/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mechanism_'+Mechanisms[f]+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W1/Peer_prediction_PMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mechanism_'+Mechanisms[f]+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W1/Peer_prediction_DMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W1/Peer_prediction_OA/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_samples.npy', S_samples)
# np.save('Samples/Effort_equilibrium/W1/Peer_prediction_PTS/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_samples.npy', S_samples)



before deviation:  0.0 2022-08-14 00:36:30.484143
before deviation:  0.02 2022-08-14 00:42:20.763561
before deviation:  0.04 2022-08-14 00:48:10.191113
before deviation:  0.06 2022-08-14 00:53:59.731219
before deviation:  0.08 2022-08-14 00:59:49.095095
before deviation:  0.1 2022-08-14 01:05:38.829530
before deviation:  0.12 2022-08-14 01:11:28.002807
before deviation:  0.14 2022-08-14 01:17:17.528471
before deviation:  0.16 2022-08-14 01:23:06.983325
before deviation:  0.18 2022-08-14 01:28:56.652280
before deviation:  0.2 2022-08-14 01:34:46.271650
before deviation:  0.22 2022-08-14 01:40:35.893099
before deviation:  0.24 2022-08-14 01:46:25.373251
before deviation:  0.26 2022-08-14 01:52:14.748524
before deviation:  0.28 2022-08-14 01:58:04.264439
before deviation:  0.3 2022-08-14 02:03:53.707577
before deviation:  0.32 2022-08-14 02:09:42.921785
before deviation:  0.34 2022-08-14 02:15:32.446102
before deviation:  0.36 2022-08-14 02:21:21.520887
before deviation:  0.38 2022-08-14 

In [ ]:
"""
Running peer prediction mechanisms in W2
"""
import datetime
w_w2 = np.array([0.19587629, 0.24054983, 0.24742268, 0.3161512])
Gamma_w2 = np.array([[0.77056673, 0.12157221, 0.08409506, 0.023766],
                 [0.09083969, 0.7351145, 0.12977099, 0.04427481],
                 [0.03326256, 0.06157113, 0.86624204, 0.03892427],
                 [0.06785509, 0.16388729, 0.09890742, 0.6693502]])
Gamma_random = np.ones((4,4))/4

"""
Switch the f functions
"""
f = 0 

m = 1000
mi = 100
n0 = 5
n = 52
T = 100
Effort_list = np.arange(0,0.95,0.02)
S_samples = np.zeros((2,len(Effort_list),n*T))
Mechanisms = ['tvd','kl','sqr','hlg']

"""
First, generate samples when everyone exerts effort xi
"""
for j,xi in enumerate(Effort_list):
    print('before deviation: ', xi, datetime.datetime.now())
    for t in range(n*T):
        para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal = [1,2,3,4])
        X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
        i = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[0,j,t] = mechanism_MMI_individual(X, f, para, i)
#         S_samples[0,j,t] = mechanism_PMI_individual(X, f, para, i)
#         S_samples[0,j,t] = mechanism_DMI_individual(X, para, i)
#         S_samples[0,j,t] = mechanism_OA_individual(X, para, i)
#         S_samples[0,j,t] = mechanism_PTS_individual(X, para, i)

"""
Then, generate samples when one agent deviates to effort xi+de
"""
for j,xi in enumerate(Effort_list):
    print('after deviation: ', xi, datetime.datetime.now())
    for t in range(n*T):
        para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal = [1,2,3,4])
        X, Y, agent_e = Report_Generator(para, [xi,xi+0.04], [n-1,1])
        i = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[1,j,t] = mechanism_MMI_individual(X, f, para, i)
#         S_samples[1,j,t] = mechanism_PMI_individual(X, f, para, i)
#         S_samples[1,j,t] = mechanism_DMI_individual(X, para, i)
#         S_samples[1,j,t] = mechanism_OA_individual(X, para, i)
#         S_samples[1,j,t] = mechanism_PTS_individual(X, para, i)

# To save space, we only store the mean and the std. Rename for different mechanisms
mean = np.average(S_samples[0], axis = 1)
mean_deviate = np.average(S_samples[1], axis = 1)
variance = np.std(S_samples[0], axis = 1)
variance_deviate = np.std(S_samples[1], axis = 1)
np.save('Samples/Effort_equilibrium/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean.npy', mean)
np.save('Samples/Effort_equilibrium/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean_deviate.npy', mean_deviate)
np.save('Samples/Effort_equilibrium/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance.npy', variance)
np.save('Samples/Effort_equilibrium/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance_deviate.npy', variance_deviate)


## 3.2 Report Strategy Deviation

### 3.2.1 Untruthful strategies

In [24]:
"""
Several examples of the heuristically chosen untruthful strategies. One can easily add more.
"""

def strategy_mixed_w1(X, s, mixed_prob):
    """
    Input: "X" is a matrix of agents' reports (all agents but i);
           "s" is the index of the untruthful strategy;
           "mixed_prob" is the probability of playing the untruthful strategy.
    Output: "X_hat" is report matrix after deviation.
    """
    X_hat = X.copy()
    index = np.where(X_hat != 0)[0]
    index = np.random.choice(index, size=int(len(index)*mixed_prob), replace=False)
    if s == 0:
        X_hat[index[np.where(X[index] == 1)[0]]] = 2
        X_hat[index[np.where(X[index] == 2)[0]]] = 3
        X_hat[index[np.where(X[index] == 3)[0]]] = 4
        X_hat[index[np.where(X[index] == 4)[0]]] = 5
    if s == 1:
        X_hat[index[np.where(X[index] == 4)[0]]] = 5
        X_hat[index[np.where(X[index] == 2)[0]]] = 1
    if s == 2: 
        X_hat[index[np.where(X[index] == 3)[0]]] = 4
    if s == 3:
        X_hat[index[np.where(X[index] == 4)[0]]] = 2
    if s == 4:
        X_hat[index[np.where(X[index] == 2)[0]]] = 3
        X_hat[index[np.where(X[index] == 4)[0]]] = 3
    if s == 5:
        X_hat[index[np.where(X[index] == 5)[0]]] = 4
    if s == 6:
        X_hat[index[np.where(X[index] == 2)[0]]] = 1
    return X_hat

def strategy_mixed_w2(X, s, mixed_prob):
    """
    Input: "X" is a matrix of agents' reports (all agents but i);
           "s" is the index of the untruthful strategy;
           "mixed_prob" is the probability of playing the untruthful strategy.
    Output: "X_hat" is report matrix after deviation.
    """
    X_hat = X.copy()
    index = np.where(X_hat != 0)[0]
    index = np.random.choice(index, size=int(len(index)*mixed_prob), replace=False) # randomly choose a subset to misreport
    if s == 0:
        X_hat[index[np.where(X[index] == 1)[0]]] = 2
        X_hat[index[np.where(X[index] == 2)[0]]] = 3
        X_hat[index[np.where(X[index] == 3)[0]]] = 4
    if s == 1:
        X_hat[index[np.where(X[index] == 1)[0]]] = 4
        X_hat[index[np.where(X[index] == 2)[0]]] = 3
    if s == 2: 
        X_hat[index[np.where(X[index] == 4)[0]]] = 3
        X_hat[index[np.where(X[index] == 2)[0]]] = 1
    if s == 3:
        X_hat[index[np.where(X[index] == 3)[0]]] = 2
        X_hat[index[np.where(X[index] == 4)[0]]] = 1
    if s == 4:
        X_hat[index[np.where(X[index] == 4)[0]]] = 2
        X_hat[index[np.where(X[index] == 1)[0]]] = 3
    if s == 5:
        X_hat[index[np.where(X[index] == 1)[0]]] = 2
    if s == 6:
        X_hat[index[np.where(X[index] == 3)[0]]] = 4
    return X_hat

### 3.2.2 Spot-checking

In [ ]:
"""
Truthfulness robustness W1
"""

w_w1 = np.array([0.612903, 0.387097])
Gamma_w1 = np.array([[0.6842, 0.2211, 0.0316, 0.0368, 0.0263], [0.0917, 0.1916, 0.05, 0.2, 0.4667]])
Gamma_random = np.ones((2,5))/5

m = 1000
mi = 100
n0 = 5
n = 52
T = 100
check_prob = 0.2 # probability of spot-checking
mixed_prob = 0.5 # probability of mixed reporting
Effort_list = [0.1, 0.4, 0.7, 1] # all agents' effort 
S_samples = np.zeros((4,n*T)) # Store the score samples before deviation. The first dimension is the effort level, the second dimension stores the samepls.
S_samples_deviate = np.zeros((4,5,n*T)) # Store the score samples after deviation. The first dimension is the effort level, the second dimension is the strategies, and the third dimension stores the samepls.

"""
Before deviation
"""
for j, xi in enumerate(Effort_list):
    for t in range(T):
        para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para, [xi], [n])
        Y_check = np.array(Y.copy())
        index = spot_check_greedy(X, check_prob)
        index_cover = np.setdiff1d(np.array(range(m)), index)
        Y_check[index_cover] = 0
        
        """
        Switch the mechanisms
        """
        S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_acc_w1(X,Y_check)
#         S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_DG_w1(X,Y_check)


"""
After deviation
"""
for j, xi in enumerate(Effort_list):
    for s in range(5): # the number of tested untruthful strategies
        for t in range(T):
            para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
            X, Y, agent_e = Report_Generator(para, [xi], [n])
            Y_check = np.array(Y.copy())
            index = spot_check_greedy(X, check_prob)
            index_cover = np.setdiff1d(np.array(range(m)), index)
            Y_check[index_cover] = 0
            for i in range(n):
                X[i] = strategy_mixed_w1(X[i], s, mixed_prob)
                
            """
            Switch the mechanisms
            """
            S_samples_deviate[j,s,n*t:n*(t+1)] = Spot_check_mechanism_acc_w1(X,Y_check)
#             S_samples_deviate[j,s,n*t:n*(t+1)] = Spot_check_mechanism_DG_w1(X,Y_check)
    print('after', xi)
    print(datetime.datetime.now())

"""
Save the samples
"""
np.save('Samples/Truthfulness_robustness/W1/Spot_checking_Acc/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)
np.save('Samples/Truthfulness_robustness/W1/Spot_checking_Acc/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples_deviate.npy', S_samples_deviate)


In [ ]:
"""
Truthfulness robustness W2
"""

w_w2 = np.array([0.19587629, 0.24054983, 0.24742268, 0.3161512])
Gamma_w2 = np.array([[0.77056673, 0.12157221, 0.08409506, 0.023766],
                 [0.09083969, 0.7351145, 0.12977099, 0.04427481],
                 [0.03326256, 0.06157113, 0.86624204, 0.03892427],
                 [0.06785509, 0.16388729, 0.09890742, 0.6693502]])
Gamma_random = np.ones((4,4))/4

m = 1000
mi = 100
n0 = 5
n = 52
T = 100
check_prob = 0.2 # probability of spot-checking
mixed_prob = 0.5 # probability of mixed reporting
Effort_list = [0.1, 0.4, 0.7, 1] # all agents' effort 
S_samples = np.zeros((4,n*T)) # Store the score samples before deviation. The first dimension is the effort level, the second dimension stores the samepls.
S_samples_deviate = np.zeros((4,5,n*T)) # Store the score samples after deviation. The first dimension is the effort level, the second dimension is the strategies, and the third dimension stores the samepls.

"""
Before deviation
"""
for j, xi in enumerate(Effort_list):
    for t in range(T):
        para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal=[1,2,3,4])
        X, Y, agent_e = Report_Generator(para, [xi], [n])
        Y_check = np.array(Y.copy())
        index = spot_check_greedy(X, check_prob)
        index_cover = np.setdiff1d(np.array(range(m)), index)
        Y_check[index_cover] = 0
        
        """
        Switch the mechanisms
        """
        S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_acc_w2(X,Y_check)
#         S_samples[j,n*t:n*(t+1)] = Spot_check_mechanism_DG_w2(X,Y_check)


"""
After deviation
"""
for j, xi in enumerate(Effort_list):
    for s in range(5): # the number of tested untruthful strategies
        for t in range(T):
            para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal=[1,2,3,4])
            X, Y, agent_e = Report_Generator(para, [xi], [n])
            Y_check = np.array(Y.copy())
            index = spot_check_greedy(X, check_prob)
            index_cover = np.setdiff1d(np.array(range(m)), index)
            Y_check[index_cover] = 0
            for i in range(n):
                X[i] = strategy_mixed_w2(X[i], s, mixed_prob)
                
            """
            Switch the mechanisms
            """
            S_samples_deviate[j,s,n*t:n*(t+1)] = Spot_check_mechanism_acc_w2(X,Y_check)
#             S_samples_deviate[j,s,n*t:n*(t+1)] = Spot_check_mechanism_DG_w2(X,Y_check)
    print('after', xi)
    print(datetime.datetime.now())

"""
Save the samples
"""
np.save('Samples/Truthfulness_robustness/W2/Spot_checking_Acc/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)
np.save('Samples/Truthfulness_robustness/W2/Spot_checking_Acc/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples_deviate.npy', S_samples_deviate)
# np.save('Samples/Truthfulness_robustness/W2/Spot_checking_DG/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples.npy', S_samples)
# np.save('Samples/Truthfulness_robustness/W2/Spot_checking_DG/strategic_m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_check_p_'+str(check_prob)+'_samples_deviate.npy', S_samples_deviate)


### 3.2.3 Peer prediction

In [ ]:
"""
Truthfulness robustness W1
"""
w_w1 = np.array([0.612903, 0.387097])
Gamma_w1 = np.array([[0.6842, 0.2211, 0.0316, 0.0368, 0.0263], [0.0917, 0.1916, 0.05, 0.2, 0.4667]])
Gamma_random = np.ones((2,5))/5

"""
Switching f
"""
f = 0

m = 1000
mi = 100
n0 = 5
n = 52
T = 3000
mixed_prob = 0.5 # probability of mixed reporting
Effort_list = [0.1, 0.4, 0.7, 1]
S_samples = np.zeros((4,T))
S_samples_deviate = np.zeros((4,5,T))

for j, xi in enumerate(Effort_list):
    for t in range(T):
        para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
        index = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[j,t] = mechanism_MMI_individual(X, f, para, index)
#         S_samples[j,t] = mechanism_PMI_individual(X, f, para, index)
#         S_samples[j,t] = mechanism_DMI_individual(X, para, index)
#         S_samples[j,t] = mechanism_OA_individual(X, para, index)
#         S_samples[j,t] = mechanism_PTS_individual(X, para, index)


for j, xi in enumerate(Effort_list):
    for s in range(5):
        for t in range(T):
            para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
            X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
            index = np.where(agent_e==1)[0][0]
            X[index] = strategy_mixed_w1(X[index], s, mixed_prob)
            
            """
            Switch the mechanisms
            """
            S_samples_deviate[j,s,t] = mechanism_MMI_individual(X, f, para, index)
#             S_samples_deviate[j,s,t] = mechanism_PMI_individual(X, f, para, index)
#             S_samples_deviate[j,s,t] = mechanism_DMI_individual(X, para, index)
#             S_samples_deviate[j,s,t] = mechanism_OA_individual(X, para, index)
#             S_samples_deviate[j,s,t] = mechanism_PTS_individual(X, para, index)
    print('after', xi)
    print(datetime.datetime.now())

    
# To save space, we only store the mean and the std. Rename for different mechanisms
mean = np.average(S_samples, axis = 1)
mean_deviate = np.average(S_samples_deviate, axis = 2)
variance = np.std(S_samples, axis = 1)
variance_deviate = np.std(S_samples_deviate, axis = 2)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean.npy', mean)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean_deviate_mixed.npy', mean_deviate)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance.npy', variance)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance_deviate_mixed.npy', variance_deviate)


In [ ]:
"""
Truthfulness robustness W2
"""
w_w2 = np.array([0.19587629, 0.24054983, 0.24742268, 0.3161512])
Gamma_w2 = np.array([[0.77056673, 0.12157221, 0.08409506, 0.023766],
                 [0.09083969, 0.7351145, 0.12977099, 0.04427481],
                 [0.03326256, 0.06157113, 0.86624204, 0.03892427],
                 [0.06785509, 0.16388729, 0.09890742, 0.6693502]])
Gamma_random = np.ones((4,4))/4

"""
Switching f
"""
f = 0

m = 1000
mi = 100
n0 = 5
n = 52
T = 3000
mixed_prob = 0.5 # probability of mixed reporting
Effort_list = [0.1, 0.4, 0.7, 1]
S_samples = np.zeros((4,T))
S_samples_deviate = np.zeros((4,5,T))

for j, xi in enumerate(Effort_list):
    for t in range(T):
        para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal=[1,2,3,4])
        X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
        index = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
        S_samples[j,t] = mechanism_MMI_individual(X, f, para, index)
#         S_samples[j,t] = mechanism_PMI_individual(X, f, para, index)
#         S_samples[j,t] = mechanism_DMI_individual(X, para, index)
#         S_samples[j,t] = mechanism_OA_individual(X, para, index)
#         S_samples[j,t] = mechanism_PTS_individual(X, para, index)


for j, xi in enumerate(Effort_list):
    for s in range(5):
        for t in range(T):
            para = Crowdsourcing(w = w_w2, Gamma_w = Gamma_w2, Gamma_s = Gamma_random, signal=[1,2,3,4])
            X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
            index = np.where(agent_e==1)[0][0]
            X[index] = strategy_mixed_w2(X[index], s, mixed_prob)
            
            """
            Switch the mechanisms
            """
            S_samples_deviate[j,s,t] = mechanism_MMI_individual(X, f, para, index)
#             S_samples_deviate[j,s,t] = mechanism_PMI_individual(X, f, para, index)
#             S_samples_deviate[j,s,t] = mechanism_DMI_individual(X, para, index)
#             S_samples_deviate[j,s,t] = mechanism_OA_individual(X, para, index)
#             S_samples_deviate[j,s,t] = mechanism_PTS_individual(X, para, index)
    print('after', xi)
    print(datetime.datetime.now())

    
# To save space, we only store the mean and the std. Rename for different mechanisms
mean = np.average(S_samples, axis = 1)
mean_deviate = np.average(S_samples_deviate, axis = 2)
variance = np.std(S_samples, axis = 1)
variance_deviate = np.std(S_samples_deviate, axis = 2)
np.save('Samples/Truthfulness_robustness/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean.npy', mean)
np.save('Samples/Truthfulness_robustness/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean_deviate_mixed.npy', mean_deviate)
np.save('Samples/Truthfulness_robustness/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance.npy', variance)
np.save('Samples/Truthfulness_robustness/W2/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_variance_deviate_mixed.npy', variance_deviate)


In [23]:
"""
Truthfulness robustness W1, for a larger range of parameters
"""
w_w1 = np.array([0.612903, 0.387097])
Gamma_w1 = np.array([[0.6842, 0.2211, 0.0316, 0.0368, 0.0263], [0.0917, 0.1916, 0.05, 0.2, 0.4667]])
Gamma_random = np.ones((2,5))/5

"""
Switching f
"""
Mechanisms = ['tvd', 'kl', 'sqr', 'hlg']
f = 0

m = 1000
mi = 100
n0 = 5
n = 52
T = 3000
mixed_prob = 0.5 # probability of mixed reporting
Effort_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # all agents' effort 
S_samples = np.zeros((8,T))
S_samples_deviate = np.zeros((8,7,T))

for j, xi in enumerate(Effort_list):
    for t in range(T):
        para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
        X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
        index = np.where(agent_e==1)[0][0]
        
        """
        Switch the mechanisms
        """
#         S_samples[j,t] = mechanism_MMI_individual(X, f, para, index)
#         S_samples[j,t] = mechanism_PMI_individual(X, f, para, index)
        S_samples[j,t] = mechanism_DMI_individual(X, para, i)
#         S_samples[j,t] = mechanism_OA_individual(X, para, i)
#         S_samples[j,t] = mechanism_PTS_individual(X, para, i)


for j, xi in enumerate(Effort_list):
    for s in range(7):
        for t in range(T):
            para = Crowdsourcing(w = w_w1, Gamma_w = Gamma_w1, Gamma_s = Gamma_random)
            X, Y, agent_e = Report_Generator(para, [xi,xi], [n-1,1])
            index = np.where(agent_e==1)[0][0]
            X[index] = strategy_mixed_w1(X[index], s, mixed_prob)
            
            """
            Switch the mechanisms
            """
#             S_samples_deviate[j,s,t] = mechanism_MMI_individual(X, f, para, index)
#             S_samples_deviate[j,s,t] = mechanism_PMI_individual(X, f, para, index)
            S_samples_deviate[j,s,t] = mechanism_DMI_individual(X, para, i)
#             S_samples_deviate[j,s,t] = mechanism_OA_individual(X, para, i)
#             S_samples_deviate[j,s,t] = mechanism_PTS_individual(X, para, i)
    print('after', xi)
    print(datetime.datetime.now())

    
# To save space, we only store the mean and the std. Rename for different mechanisms
mean = np.average(S_samples, axis = 1)
mean_deviate = np.average(S_samples_deviate, axis = 2)
std = np.std(S_samples, axis = 1)
std_deviate = np.std(S_samples_deviate, axis = 2)
# np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'mechanism'+Mechanisms[f]+'_mean_larger.npy', mean)
# np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'mechanism'+Mechanisms[f]+'_mean_deviate_larger.npy', mean_deviate)
# np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'mechanism'+Mechanisms[f]+'_std_larger.npy', std)
# np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_MMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'mechanism'+Mechanisms[f]+'_std_deviate_larger.npy', std_deviate)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_DMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean_larger.npy', mean)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_DMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_mean_deviate_larger.npy', mean_deviate)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_DMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_std_larger.npy', std)
np.save('Samples/Truthfulness_robustness/W1/Peer_prediction_DMI/m'+str(m)+'_mi'+str(mi)+'_n0'+str(n0)+'_T'+str(T)+'_std_deviate_larger.npy', std_deviate)


after 0.2
2022-08-16 17:39:11.134627
after 0.3
2022-08-16 17:59:32.394611
after 0.4
2022-08-16 18:19:54.673390
after 0.5
2022-08-16 18:40:17.033655
after 0.6
2022-08-16 19:00:38.698707
after 0.7
2022-08-16 19:20:59.121688
after 0.8
2022-08-16 19:41:20.212646
after 0.9
2022-08-16 20:01:41.443252
